# Lecture Notes: Boolean Abstraction and SMT Solvers

<br>

<img src="./alr-images/1100.png" width="500" style="display: block; margin: auto;">

<br>

## Motivation

Up to now, we've studied decision procedures for **conjunctive formulas** in various first-order theories. To handle disjunctions, we often mentioned converting to **disjunctive normal form (DNF)**. However:

- **Converting to DNF causes exponential blowup**.
- Unlike CNF, DNF lacks efficient satisfiability-preserving transformations.
- We need a better approach for formulas involving rich Boolean structure **and** first-order theories.

---

## SMT Solvers (Satisfiability Modulo Theories)

<br>

<img src="./alr-images/1101.png" width="500" style="display: block; margin: auto;">

<br>

SMT solvers answer whether a formula is **satisfiable modulo some theory** (e.g., integers, rationals, arrays).

### Key Insight:
Use **SAT solvers** to handle the Boolean structure, and **theory solvers** to handle the theory-specific atoms.

### Core Idea:

<br>

<img src="./alr-images/1102.png" width="500" style="display: block; margin: auto;">

<br>

1. Convert the first-order formula into a **Boolean abstraction**, which overapproximates satisfiability.
2. Query the SAT solver on the boolean abstraction, if UNSAT, then also UNSAT modulo theory.
3. If SAT, ask the **theory solver**: is this assignment consistent with the theory?
   - If **yes** → satisfiable.
   - If **no** → generate a **theory conflict clause** to block that assignment and repeat.

This is the foundation of the **DPLL(T) framework**.

---

## Boolean Abstraction (a.k.a. Boolean Skeleton)

<br>

<img src="./alr-images/1103.png" width="500" style="display: block; margin: auto;">

<br>

Given a formula built from theory atoms and Boolean connectives:

<br>

<img src="./alr-images/1104.png" width="500" style="display: block; margin: auto;">

<br>

<br>

<img src="./alr-images/1105.png" width="500" style="display: block; margin: auto;">

<br>

### Step 1: Replace each **atomic formula** with a fresh propositional variable.
- Example:
  ```
  F = (x = y) ∧ (y = z) ∧ (x < z ∨ x = z)
  ```
  - Let:
    - p₁ ≡ (x = y)
    - p₂ ≡ (y = z)
    - p₃ ≡ (x < z)
    - p₁ reused for (x = z)
  - Boolean abstraction:
    ```
    B(F) = p₁ ∧ p₂ ∧ (p₃ ∨ p₁)
    ```

### Step 2: Define **B inverse** mapping:
- Allows converting Boolean models back to the original theory:
  - B⁻¹(p₂ ∨ ¬p₁) = (y = z) ∨ (x ≠ y)

---

## Why Boolean Abstraction Is an Over-Approximation

<br>

<img src="./alr-images/1106.png" width="500" style="display: block; margin: auto;">

<br>

If the **Boolean abstraction** is **unsatisfiable**, then the original formula is **unsatisfiable** in the theory.

However, if the Boolean abstraction is **satisfiable**, the original formula **might not be**.

### Example:
Formula:
```
x = z ∧ y = z ∧ (x < z ∨ x \ne z)
```
Boolean abstraction:
```
p₁ ∧ p₂ ∧ (p₃ ∨ p₁)
```

SAT solver finds:
```
p₁ = true, p₂ = true, p₃ = true
```
→ B⁻¹ = `x = z ∧ y = z ∧ x < z` → **unsatisfiable** in theory of rationals/integers

Thus, we need to **block this assignment** from reoccurring.

---

## Theory Conflict Clauses

<br>

<img src="./alr-images/1107.png" width="500" style="display: block; margin: auto;">

<br>

When a Boolean assignment is **inconsistent with the theory**, we generate a **blocking clause** (aka theory conflict clause) to rule it out.

### Process:
1. Get assignment from SAT solver (e.g., p₁ ∧ p₂ ∧ p₃).
2. Ask theory solver: is this theory-consistent?
3. If not:
   - Add a conflict clause like `¬(p₁ ∧ p₂ ∧ p₃)` → e.g., `(¬p₁ ∨ ¬p₂ ∨ ¬p₃)`
   - This strengthens the Boolean abstraction for future iterations.

---

## Two Approaches to SMT Solving

### 1. **Offline / Eager SMT Solving**
- Treats the **SAT solver as a black box**.
- The SAT solver proposes assignments; the theory solver checks consistency.
- Adds conflict clauses externally.

### 2. **Online / Lazy SMT Solving**
- **Integrates** the theory solver into the **CDCL (Conflict-Driven Clause Learning)** loop.
- More efficient and avoids redundant work.
- Treated like a module inside the SAT solver's decision engine.

---

## Summary

| Component         | Role                                                                 |
|------------------|----------------------------------------------------------------------|
| **Boolean Abstraction** | Translates theory formula to propositional logic (over-approximate) |
| **B Inverse**     | Maps propositional assignments back to theory formulas               |
| **SAT Solver**    | Handles Boolean structure; proposes assignments                     |
| **Theory Solver** | Validates theory consistency of assignments                         |
| **Conflict Clause** | Blocks theory-inconsistent Boolean models                         |

This approach avoids DNF conversion and efficiently combines Boolean reasoning with theory reasoning.

---

## Next Up
We'll explore the **offline/eager SMT method**, followed by **online/lazy integration** inside CDCL solvers.

